In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.profiler import profile, record_function, ProfilerActivity
from numpy import log 

torch.manual_seed(123)

In [2]:

# Generalisable NN
torch.manual_seed(32)

class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, output_dim, layer_dim, num_hidden, reg_param, reg_power, ran_seed):
        super().__init__()

        # Define size of input and output layers
        self.input_dim = input_dim
        self.output_dim = output_dim

        # Define size of hidden layers (number of neurons)
        self.layer_dim = layer_dim

        # Define how many hidden layers
        self.num_hidden = num_hidden

        # Define activation function of neurons 
        self.activation = nn.Tanh()

        # Define regularisation parameter
        self.reg_param = reg_param

        # Defin regularisation exponent power
        self.reg_power = reg_power

        # Define random seed to be used
        self.ran_seed = ran_seed

        # Build architectyre
        self.input_layer = nn.Linear(self.input_dim, self.layer_dim)
        self.hidden_layers = nn.ModuleList([nn.Linear(self.layer_dim, self.layer_dim) for _ in range(num_hidden-1)])
        self.output_layer = nn.Linear(self.layer_dim, self.output_dim)
        self.layers = nn.ModuleList([self.input_layer, *self.hidden_layers, self.output_layer])

        # Initialise weights
        self.initialise()

    def forward(self, x):
    # Function defines the forward pass of the network
        for _, f in enumerate(self.layers[:-1]):
            x = self.activation(f(x))
        x = self.output_layer(x)
        return x
    
    def initialise(self):
    # Function defines initilaising the unfrozen weights of the network using Xavier (Glorot) initialisiation
        torch.manual_seed(self.ran_seed)
        def init_weights(m):
            if type(m) == nn.Linear and m.weight.requires_grad and m.bias.requires_grad:
                g = nn.init.calculate_gain('tanh')
                #torch.nn.init.xavier_uniform_(m.weight, gain=g)
                torch.nn.init.xavier_normal_(m.weight, gain=g)
                m.bias.data.fill_(0)

        self.apply(init_weights)

    def regularization(self):
    # Function defines regularisation
        reg_loss = 0
        for name, param in self.named_parameters():
            if 'weight' in name:
                reg_loss = reg_loss + torch.norm(param, self.reg_power)
        return self.reg_param * reg_loss
    
    

In [4]:
# Define the PINN class
class PINN(nn.Module):
    def __init__(self, balancing_parameter, input_dim, output_dim, layer_dim, num_hidden, reg_param, reg_power, ran_seed):
        super(PINN, self).__init__()

        # Initialize the NeuralNetwork for intensity and absorption coefficient
        self.intensity_network = NeuralNetwork(input_dim, output_dim, layer_dim, num_hidden, reg_param, reg_power, ran_seed)
        self.emission_network = NeuralNetwork(input_dim, output_dim, layer_dim, num_hidden, reg_param, reg_power, ran_seed)
        self.alpha_network = NeuralNetwork(input_dim, output_dim, layer_dim, num_hidden, reg_param, reg_power, ran_seed)

        self.balancing_parameter = balancing_parameter


    def compute_pde_residual(self, input_distance):
            '''
            Enforces PDE via automatic differentiation.
            Uses NN to predict intensity and absorption, and calculates dI/ds.
            '''
            input_distance.requires_grad = True

            # Predict intensity and absorption coefficient
            pred_intensity = self.intensity_network(input_distance)
            emission = self.emission_network(input_distance)
            alpha = self.alpha_network(input_intensity)

            # Compute derivative dI/ds using autograd
            dintensity = torch.autograd.grad(pred_intensity.sum(), input_distance, create_graph=True)[0]

            # PDE residual: dI/ds + alpha * I = 0
            residual = dintensity + alpha * pred_intensity - emission

            return residual.reshape(-1,)
    
    def compute_loss(self, dist_bound, int_bound, input_distance, input_intensity, verbose=True):
        '''
        Computes loss using boundary conditions, physics residuals, and data loss.
        '''
        # Predict intensity at boundaries
        #pred_initial_intensity = self.intensity_network(dist_bound[0])
        pred_boundary_intensity = self.intensity_network(dist_bound)

        # Compute PDE residuals (physics enforcement)
        physics_residual = self.compute_pde_residual(input_distance)

        # **Hard enforce boundary conditions**
        #boundary_loss = torch.mean((pred_initial_intensity - int_bound[0])**2)

        boundary_loss = torch.mean((pred_boundary_intensity - int_bound)**2)

        # Compute data residuals inside the slab
        pred_intensity = self.intensity_network(input_distance)
        data_residual = input_intensity - pred_intensity

        # Compute loss terms (L2 loss)
        physics_loss = torch.mean(abs(physics_residual)**2)
        data_loss = torch.mean(abs(data_residual)**2)

        # Add regularization terms
        intensity_reg = self.intensity_network.regularization()
        emission_reg = self.emission_network.regularization()
        

        # Total loss (weighted combination)
        total_loss = torch.log(self.balancing_parameter * (data_loss+boundary_loss) + physics_loss + intensity_reg + emission_reg)
        
        if verbose:
            print(f"Total Loss: {total_loss.item():.4f} | PDE Loss: {torch.log10(physics_loss).item():.4f} | "
                f"Boundary Loss: {torch.log10(boundary_loss).item():.4f} | Data Loss: {torch.log10(data_loss).item():.4f}")
        
        return total_loss
    

# Define the function for the analytical solution
def analytical_solution(input_distance, emission_value,alpha_value, I0=0.0):
    return emission_value/alpha_value +(I0 - emission_value/alpha_value) * torch.exp(-alpha_value * input_distance)

def plot_results(model, input_distance, analytical_solution, emission_value):
    # Predict intensity from the trained model
    pred_intensity = model.intensity_network(input_distance).detach()

    # Compute the analytical solution using the true alpha value
    anal_intensity = analytical_solution(input_distance, emission_value)

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(input_distance.detach().numpy(), pred_intensity.detach().numpy(), label='Predicted Intensity', color='b')
    plt.plot(input_distance.detach().numpy(), anal_intensity.detach().numpy(), label='Analytical Intensity', color='r', linestyle='--')
    plt.xlabel('Distance (s)')
    plt.ylabel('Intensity')
    plt.legend()
    plt.title('Intensity Prediction vs Analytical Solution')
    plt.show()

    pred_emission = model.emission_network(input_distance).detach().numpy().flatten()

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(input_distance.detach().numpy(), pred_emission, label='Predicted emission', color='b')
    plt.axhline(y=emission_value, color='r', linestyle='--', label=f'True emission ({emission_value})')
    plt.xlabel('Distance (s)')
    plt.ylabel('emission (j)')
    plt.legend()
    plt.title('Predicted vs Analytical emission')
    plt.show()
    pred_alpha = model.emission_network(input_distance).detach().numpy().flatten()

    # Plot results
    plt.figure(figsize=(10, 6))
    plt.plot(input_distance.detach().numpy(), pred_alpha, label='Predicted emission', color='b')
    plt.axhline(y=alpha_value, color='r', linestyle='--', label=f'True emission ({emission_value})')
    plt.xlabel('Distance (s)')
    plt.ylabel('emission (j)')
    plt.legend()
    plt.title('Predicted vs Analytical emission')
    plt.show()

def fit(model, optimizer, dist_bound, int_bound, input_distance, input_intensity, epochs):
    '''
    Fits the PINN model using Adam optimization.
    '''
    history = []  # Store loss values

    for epoch in range(epochs):
        def closure():
            optimizer.zero_grad()
            loss = model.compute_loss(dist_bound, int_bound, input_distance, input_intensity)
            loss.backward()
            history.append(loss.item())  # Store loss at each epoch
            return loss

        optimizer.step(closure)

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {history[-1]:.6f}")

    print(f"Final Loss: {history[-1]:.6f}")

def calculate_relative_l2_error(model, input_distance, analytical_solution, emission_value):
    """
    Calculates the relative L2 error between the predicted intensity and the analytical solution as a percentage.
    """
    # Predict intensity from the trained model
    pred_intensity = model.intensity_network(input_distance).detach().flatten()

    # Compute the analytical solution using the true alpha value
    anal_intensity = analytical_solution(input_distance, emission_value).flatten()

    # Compute the L2 norm of the difference between predicted and analytical solutions
    numerator = torch.norm(pred_intensity - anal_intensity, p=2)

    # Compute the L2 norm of the analytical solution
    denominator = torch.norm(anal_intensity, p=2)

    # Compute the relative L2 error as a percentage
    relative_l2_error = (numerator / denominator) * 100

    return relative_l2_error.item()

# ==== Initialize Model and Training ==== 
balancing_parameter = 5.0  # Adjust this if needed
input_dim = 1
output_dim = 1
layer_dim = 20
num_hidden = 6
reg_param = 0.001
reg_power = 1
ran_seed = 32

model = PINN(balancing_parameter, input_dim, output_dim, layer_dim, num_hidden, reg_param, reg_power, ran_seed)

optimizer_Adam = optim.Adam(
    list(model.intensity_network.parameters()) + list(model.emission_network.parameters()),
    lr=1e-3,  # Adam optimizer typically uses learning rates like this
    betas=(0.5, 0.999),  # Lower momentum for better convergence
    eps=1e-8,  # Small value to prevent division by zero
    weight_decay=0  # LBFGS doesn't typically use weight decay, so keep it 0
)

# Define LBFGS optimizer
optimizer_LBFGS = optim.LBFGS(
    list(model.intensity_network.parameters()) + list(model.emission_network.parameters()),
    lr=1e-3,
    max_iter=1000,
    max_eval=50000,
    history_size=150,
    line_search_fn="strong_wolfe",
    tolerance_change=np.finfo(float).eps
)

# ==== Training Data (Example Boundaries and Grid Points) ==== 
I0 = 0.0  # Initial intensity
emission_value = 2.0  # Absorption coefficient
alpha_value = 0.5

dist_bound = torch.tensor([[0.0], [10.0]])  # Slab distance from 0 to 10
int_bound = torch.tensor([[I0], [emission_value/alpha_value +(I0 - emission_value/alpha_value)*torch.exp(-alpha_value*10)]])  # Intensity at boundaries

x_min = dist_bound.min().item()
x_max = dist_bound.max().item()
input_distance = torch.linspace(x_min, x_max, 100).view(-1, 1)  # Interior grid

input_intensity = analytical_solution(input_distance, emission_value, alpha_value)

# Train the model
fit(model, optimizer_LBFGS, dist_bound, int_bound, input_distance, input_intensity, epochs=50)

# Calculate relative L2 error as a percentage
relative_l2_error = calculate_relative_l2_error(model, input_distance, analytical_solution, emission_value)
print(f"Relative L2 Error: {relative_l2_error:.6f}%")

# Plot the results **only once** after training
plot_results(model, input_distance, analytical_solution,emission_value, alpha_value)


TypeError: exp(): argument 'input' (position 1) must be Tensor, not float